In [1]:
from skyfield.api import N, W, S, E, wgs84
from skyfield.api import load
from skyfield.api import Angle
import math
import numpy as np

In [2]:
ts = load.timescale()
planets = load('de405.bsp')

In [3]:
sun = planets['sun']
earth = planets['earth']

In [4]:
def createAngle(degrees, minutes, sign):
    if (sign == None): 
        sign = 1
    return (degrees + minutes/60.0) * sign

def toDegreesAndMinutes(angle):
    sign = 1
    if (angle < 0): sign = -1
    angle = angle*sign
    degrees = math.floor(angle)
    minutes = round((angle - degrees) * 60.0, 2)
    return degrees*sign, minutes
    
def angleToString(angle):
    degrees, minutes = toDegreesAndMinutes(angle)
    return "{} degrees; {} minutes".format(degrees, minutes)

def angleToStringDelta(angle):
    degrees, minutes = toDegreesAndMinutes(angle)
    if (degrees == 0):
        return "{} minutes".format(minutes)
    else:
        return "{} degrees; {} minutes".format(degrees, minutes)

In [5]:
def compute_GHA_dec(year, month, day, hour, minute, second):
    point = wgs84.subpoint(earth.at(ts.utc(year,month,day,hour,minute,second)).observe(sun))
    if (point.longitude.dms()[0]>0):
        # East Longitude.  We need to do 360 - the longitude to get GHA.
        return 360.0-point.longitude.degrees, point.latitude.degrees
    else:
        # West Longitude.  Skyfield represents W longitudes as -, convert to positive with W constant (-1)
        return point.longitude.degrees*W, point.latitude.degrees

In [6]:
def compute_LHA(GHA, dr_lon):
    # Compute LHA.
    if (dr_lon < 0):
        # Western hemisphere.  LHA = GHA - a_lon
        LHA = GHA - (dr_lon*W)
    else:
        # Easter hemisphere.  LHA = GHA + a_lon
        LHA = GHA + (dr_lon*E)
        
    return LHA

In [7]:
def sight_reduction(dec, lat, LHA):
    dec_rads = math.radians(dec)
    lat_rads = math.radians(lat)
    LHA_rads = math.radians(LHA)
    
    if ((dec_rads < 0) and (lat_rads < 0)):
        # If dec and lat are same hemisphere (i.e. SAME), then both values should be positive.
        # here there are both south (so negative) take the absolute value so that they are both positive.
        dec_rads = abs(dec_rads)
        lat_rads = abs(lat_rads)
    elif ((dec_rads > 0) and (lat_rads < 0)):
        # if dec and lat different hemispheres (i.e. CONTRARY), then dec should be made negative.
        # here we have a south latitude and north declination, so shift the negative to dec.
        dec_rads = dec_rads *-1
        lat_rads = lat_rads *-1
    
    # Hc
    sin_Hc_rads = math.sin(lat_rads)*math.sin(dec_rads) + math.cos(lat_rads)*math.cos(dec_rads)*math.cos(LHA_rads) 
    Hc_rads = math.asin(sin_Hc_rads)
    Hc = math.degrees(Hc_rads)
    
    #Z       
    cos_Z_rads_num = math.sin(dec_rads) - math.sin(Hc_rads)*math.sin(lat_rads)
    cos_Z_rads_den = math.cos(Hc_rads) * math.cos(lat_rads) 
    cos_Z_rads = cos_Z_rads_num / cos_Z_rads_den
    
    # Clip values that are > 1 or < -1  (usually introduced due to rounding error)
    if (cos_Z_rads > 1):
        print("cos_Z_rads > 1 {}".format(cos_Z_rads))
        cos_Z_rads = 1
    elif (cos_Z_rads < -1):
        print("cos_Z_rads < 1 {}".format(cos_Z_rads))
        cos_Z_rads = -1
    
    Z = math.degrees(math.acos(cos_Z_rads))
    
    return Hc, Z

In [8]:
Off = 1
On = -1

LowerLimb = 0
UpperLimb = 1

app_alt_oct_mar = np.array([[9, 30],[9, 45],[9, 56],[10, 8],[10, 20],[10, 33],[10, 46],[11, 0],[11, 15],[11, 30],[11, 45],[12, 1],[12, 18],[12, 36],[12, 54],[13, 14],[13, 34],[13, 55],[14, 17],[14, 41],[15, 5],[15, 31],[15, 59],[16, 27],[16, 58],[17, 30],[18, 5],[18, 41],[19, 20],[20, 2],[20, 46],[21, 34],[22, 25],[23, 20],[24, 20],[25, 24],[26, 34],[27, 50],[29, 13],[30, 44],[32, 24],[34, 15],[36, 17],[38, 34],[41, 6],[43, 56],[47, 7],[50, 43],[54, 46],[59, 21],[64, 28], [70, 10],[76, 24],[83, 5],[90, 0]])
app_alt_apr_sept = np.array([[9, 39], [9, 50], [10, 2], [10, 14], [10, 27], [10, 40], [10, 53], [11, 7], [11, 22], [11, 37], [11, 53], [12, 10], [12, 27], [12, 45], [13, 4], [13, 24], [13, 44], [14, 6], [14, 29], [14, 53], [15, 18], [15, 45], [16, 13], [16, 43], [17, 14], [17, 47], [18, 23], [19, 0], [19, 41], [20, 24], [21, 10], [21, 59], [22, 52], [23, 49], [24, 51], [25, 58], [27, 11], [28, 31], [29, 58], [31, 33], [33, 18], [35, 15], [37, 24], [39, 48], [42, 28], [45, 29], [48, 52], [51, 41], [56, 59], [61, 50], [67, 15], [73, 14], [79, 42], [86, 21], [90, 0]])


def sun_alt_corr(sun_apt_alt, Ha, limb_adjust_base):
    for i in range(sun_apt_alt.shape[0]):
        degrees = createAngle(sun_apt_alt[i, 0], sun_apt_alt[i, 1], None)
        if (degrees > Ha):
            break
    
    return round (limb_adjust_base + 0.1 * (i-1), 1)
    
def alt_corr(Ha, limb, month):
    if ((month >= 10) or (month <= 3)): #Oct - March
        limb_adjust_base = 10.8 # LowerLimb for Oct-Mar
        if (limb == UpperLimb):
            limb_adjust_base = -21.5
            
        corr_min = sun_alt_corr(app_alt_oct_mar, Ha, limb_adjust_base)
    else:
        
        limb_adjust_base = 10.6 # LowerLimb for Apr-Sept
        if (limb == UpperLimb):
            limb_adjust_base = -21.2
        corr_min = sun_alt_corr(app_alt_apr_sept, Ha, limb_adjust_base)
    
    return createAngle(0, corr_min, None)

def compute_Ha(Hs, ic, heightEyeFt):
    DIP_min = round(0.97 * math.sqrt(heightEyeFt), 1)
    DIP = createAngle(0, DIP_min, None)
    return Hs + ic - DIP

In [9]:
def compute_Zn(Z, lat, LHA):
    #Zn
    Zn = Z
    if (lat > 0):
        # Northern latitude.
        if (LHA < 180):
            Zn = 360 - Z
    else:
        # Souther latitude.
        if (LHA > 180):
            Zn = 180-Z
        else:
            Zn = 180+Z
            
    if (Zn == 360):
        Zn = 0
    return Zn

In [10]:
def compute_a(Ho, Hc):
    if (Ho > Hc):
        return Ho - Hc, 'T'
    if (Ho < Hc):
        return Hc - Ho, 'A'

In [11]:
def compute_lat_from_Ho(dec, Ho, dr_lat):
    looking = N
    if (dr_lat > 23.5):
        looking = S
    elif (dr_lat < -23.5):
        looking = N
    elif (dec < dr_lat):
        looking = S
    else:
        looking = N
    
    zenith = 90 - Ho
    return dec - zenith*looking

In [12]:
def compute_position(year, month, day, hour, minute, second, dr_lat, dr_lon, Hs, limb, ic, heightEyeFt):
    Ha = compute_Ha (Hs, ic, heightEyeFt)
    #print("Hs: {}".format(angleToString(Hs)))
    #print("Ha: {}".format(angleToString(Ha)))

    alt_correction = alt_corr(Ha, limb, month)
    Ho = Ha + alt_correction
    #print("alt_correction: {}".format(angleToString(alt_correction)))
    #print("Ho: {}".format(angleToString(Ho)))

    GHA, dec = compute_GHA_dec(year, month, day, hour, minute, second)
    #print("GHA: {}".format(angleToString(GHA)))
    #print("dec: {}".format(angleToString(dec)))

    a_lat = round(dr_lat)
    LHA = compute_LHA(dr_lon, GHA)

    #print("a_lat: {}".format(angleToString(a_lat)))
    #print("LHA: {}".format(angleToString(LHA)))

    Hc, Z = sight_reduction(dec, a_lat, LHA)
    Z = round(Z, 0)
    Zn = compute_Zn(Z, a_lat, LHA)
    #print("Hc: {}".format(angleToString(Hc)))
    #print("Z: {}".format(angleToString(Z)))
    #print("Zn: {}".format(angleToString(Zn)))

    a, ta = compute_a(Ho, Hc)
    #print("a: {} {} {}".format(angleToString(a), ta, round(Zn, 0)))
    
    lat = compute_lat_from_Ho(dec, Ho, dr_lat)
    #print("lat: {}".format(angleToString(lat)))
    
    return Ho, LHA, Hc, Zn, a, ta, lat

In [13]:
data = [
    [1978, 7, 25, 23, 4, 0, createAngle(56, 2, N), createAngle(164, 30, W), createAngle(54, 5, None), LowerLimb, createAngle(0, 2, On), 9],
    [1978, 10, 27, 22, 49, 0, createAngle(10, 0, S), createAngle(166, 15, W), createAngle(87, 20, None), UpperLimb, createAngle(0, 2, Off), 25],
    [1978, 10, 25, 6, 48, 0, createAngle(35, 54, N), createAngle(74, 2, E), createAngle(41, 30.5, None), LowerLimb, createAngle(0, 4, Off), 10],
    [1981, 3, 27, 13, 7, 0, createAngle(37, 40, N), createAngle(15, 24, W), createAngle(54, 41.3, None), LowerLimb, createAngle(0, 2, Off), 9],
    [1981, 3, 28, 7, 48, 0, createAngle(22, 38, N), createAngle(64, 17, E), createAngle(71, 9.8, None), LowerLimb, createAngle(0, 2.2, On), 9],
    [1978, 10, 26, 0, 51, 0, createAngle(27, 6, S), createAngle(163, 16, E), createAngle(74, 59.8, None), LowerLimb, createAngle(0, 1.4, Off), 9],
    [1978, 7, 24, 17, 48, 0, createAngle(28, 44, S), createAngle(85, 17, W), createAngle(42, 51.2, None), UpperLimb, createAngle(0, 1.5, On), 9],
    [1978, 7, 25, 20, 50, 0, createAngle(44, 10, N), createAngle(131, 0, W), createAngle(65, 22.5, None), LowerLimb, createAngle(0, 2, On), 9],
    [1981, 3, 28, 21, 51, 0, createAngle(16, 40, S), createAngle(146, 30, W), createAngle(69, 44.2, None), LowerLimb, createAngle(0, 2.5, Off), 12],
    [1978, 10, 25, 10, 4, 0, createAngle(34, 29, N), createAngle(25, 0, E), createAngle(43, 11.7, None), LowerLimb, createAngle(0, 1.5, Off), 14]
]

expected = [
    [0, createAngle(54, 15.4, None), createAngle(53, 34.4, None), createAngle(0, 41, None), "T", 180, createAngle(55, 19, N)],
    [0, createAngle(87, 1, None), createAngle(87, 5.5, None), createAngle(0, 4.6, None), "A", 180, createAngle(9, 55.4, S)],
    [360, createAngle(41, 46.6, None), createAngle(42, 0.2, None), createAngle(0, 13.6, None), "A", 180, createAngle(36, 13.6, N)],
    [0, createAngle(54, 55.9, None), createAngle(54, 41.4, None), createAngle(0, 14.5, None), "T", 180, createAngle(37, 45.5, N)],
    [360, createAngle(71, 20.6, None), createAngle(69, 59.7, None), createAngle(1, 20.9, None), "T", 180, createAngle(21, 39.1, N)],
    [360, createAngle(75, 14.2, None), createAngle(75, 15.4, None), createAngle(0, 1.2, None), "A", 0, createAngle(27, 1.2, S)],
    [0, createAngle(42, 30, None), createAngle(41, 10, None), createAngle(1, 20, None), "T", 0, createAngle(27, 40, S)],
    [0, createAngle(65, 33.1, None), createAngle(65, 35.6, None), createAngle(0, 2.5, None), "A", 180, createAngle(44, 2.5, N)],
    [0, createAngle(69, 59.1, None), createAngle(69, 46.6, None), createAngle(0, 12.5, None), "T", 0, createAngle(16, 47.5, S)],
    [360, createAngle(43, 24.8, None), createAngle(43, 57.3, None), createAngle(0, 32.5, None), "A", 180, createAngle(34, 32.5, N)]
]

threshold = createAngle(0, 0.2, None)
LHA_threshold = createAngle(0, 30, None)  # Answers from book use a method of rounding to nearest degree.
success = True

for i in range (10):
    year, month, day, hour, minute, second, dr_lat, dr_lon, Hs, limb, ic, heightEyeFt = data[i]
    Ho, LHA, Hc, Zn, a, ta, lat = compute_position (year, month, day, hour, minute, second, dr_lat, dr_lon, Hs, limb, ic, heightEyeFt)
    LHA_expected, Ho_expected, Hc_expected, a_expected, ta_expected, Zn_expected, lat_expected = expected[i]

    LHA_delta = abs(LHA-LHA_expected)
    LHA_passed = (LHA_delta<LHA_threshold)
    print ("{}: LHA={}; delta={}".format("PASSED" if LHA_passed else "FAILED", angleToString(LHA), angleToStringDelta(LHA_delta)))

    
    Ho_delta = abs(Ho-Ho_expected)
    Ho_passed = (Ho_delta<threshold)
    print ("{}: Ho={}; delta={}".format("PASSED" if Ho_passed else "FAILED", angleToString(Ho), angleToStringDelta(Ho_delta)))
    
    
    Hc_delta = abs(Hc-Hc_expected)
    Hc_passed = (Hc_delta<threshold)
    print ("{}: Hc={}; delta={}".format("PASSED" if Hc_passed else "FAILED", angleToString(Hc), angleToStringDelta(Hc_delta)))
    
    a_delta = abs(a-a_expected)
    a_passed = ((a_delta<threshold) and (ta == ta_expected))
    print ("{}: a={} {}; delta={}".format("PASSED" if a_passed else "FAILED", angleToString(a), ta, angleToStringDelta(a_delta)))
    if (ta != ta_expected):
        print("MISMATCH on T/A:  ta={}; ta_expected={}".format(ta, ta_expected))
   
    Zn_delta = abs(Zn-Zn_expected)
    Zn_passed = (Zn_delta<threshold)
    print ("{}: Zn={}; delta={}".format("PASSED" if Zn_passed else "FAILED", angleToString(Zn), angleToStringDelta(Zn_delta)))
    
    lat_delta = abs(lat-lat_expected)
    lat_passed = (lat_delta<threshold)
    print ("{}: lat={}; delta={}".format("PASSED" if lat_passed else "FAILED", angleToString(lat), angleToStringDelta(lat_delta)))
    print ("==========================")
    
    if (success == True):
        success = LHA_passed and Ho_passed and Hc_passed and a_passed and Zn_passed and lat_passed

    
print("Overall result: {}".format("PASSED" if (success==True) else "FAILED! Check above results."))



PASSED: LHA=0 degrees; 7.02 minutes; delta=7.02 minutes
PASSED: Ho=54 degrees; 15.4 minutes; delta=0.0 minutes
PASSED: Hc=53 degrees; 34.28 minutes; delta=0.12 minutes
PASSED: a=0 degrees; 41.12 minutes T; delta=0.12 minutes
PASSED: Zn=180 degrees; 0.0 minutes; delta=0.0 minutes
PASSED: lat=55 degrees; 18.88 minutes; delta=0.12 minutes
PASSED: LHA=0 degrees; 1.26 minutes; delta=1.26 minutes
PASSED: Ho=87 degrees; 1.0 minutes; delta=0.0 minutes
PASSED: Hc=87 degrees; 5.44 minutes; delta=0.06 minutes
PASSED: a=0 degrees; 4.44 minutes A; delta=0.16 minutes
PASSED: Zn=180 degrees; 0.0 minutes; delta=0.0 minutes
PASSED: lat=-9 degrees; 55.56 minutes; delta=0.16 minutes
PASSED: LHA=359 degrees; 59.16 minutes; delta=0.84 minutes
PASSED: Ho=41 degrees; 46.6 minutes; delta=0.0 minutes
PASSED: Hc=42 degrees; 0.08 minutes; delta=0.12 minutes
PASSED: a=0 degrees; 13.48 minutes A; delta=0.12 minutes
PASSED: Zn=180 degrees; 0.0 minutes; delta=0.0 minutes
PASSED: lat=36 degrees; 13.48 minutes; delta=